<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_teachability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

# TeachableAgent와 채팅하기

LLM 기반의 대화형 어시스턴트는 사용자와의 현재 채팅을 기억할 수 있으며, 채팅 중에 사용자가 어시스턴트에게 가르치는 내용을 맥락에 맞게 학습하는 모습을 보여줄 수도 있습니다. 하지만 채팅이 종료되거나 단일 채팅이 너무 길어져 LLM이 효과적으로 처리할 수 없게 되면 이러한 기억과 학습은 손실됩니다. 이후 채팅에서 사용자는 필요한 지침을 계속해서 반복해야 합니다.

'티처블 에이전트'는 채팅 경계를 넘나드는 사용자 교육을 장기 메모리(벡터 데이터베이스)에 지속함으로써 이러한 한계를 해결합니다. 메모리는 각 채팅이 끝날 때 디스크에 저장되고 다음 채팅이 시작될 때 디스크에서 로드됩니다. 모든 메모리를 컨텍스트 창에 복사하여 공간을 많이 차지하는 대신, 필요에 따라 개별 메모리(메모라고 함)를 컨텍스트에서 검색합니다. 따라서 사용자는 티칭 가능한 상담원에게 자주 사용하는 사실과 기술을 한 번만 가르치고 이후 채팅에서 기억하도록 할 수 있습니다.

메모 저장 및 검색에 대한 결정을 내릴 때 `TeachableAgent`는 `TextAnalyzerAgent`의 인스턴스를 호출하여 여러 가지 방법으로 텍스트를 분석합니다. 이렇게 하면 상대적으로 적은 수의 토큰과 관련된 추가 LLM 호출이 추가됩니다. 이러한 호출은 사용자가 응답을 기다리는 시간에 몇 초를 추가할 수 있습니다.

이 노트북은 `TeachableAgent`가 사용자로부터 사실, 선호도, 기술을 어떻게 학습할 수 있는지 보여줍니다. TeachableAgent`와 직접 채팅하려면 [chat_with_teachable_agent.py](../test/agentchat/chat_with_teachable_agent.py)를 실행하세요.

## 요구 사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 [teachable] 옵션을 설치하세요.
```bash
pip install "pyautogen[teachable]"
```

In [ ]:
%%capture --no-stderr
# %pip install "pyautogen[teachable]

## API 엔드포인트 설정

`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) 함수는 환경 변수 또는 json 파일에서 구성 목록을 로드합니다.

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print(config_list[0]["model"])

gpt-4


먼저 유효한 json 문자열이어야 하는 환경 변수 "OAI_CONFIG_LIST"를 찾습니다. 해당 변수를 찾을 수 없으면 "OAI_CONFIG_LIST"라는 이름의 json 파일을 찾습니다. 이 파일은 모델별로 구성을 필터링합니다(다른 키로도 필터링할 수 있음). 이 특정 필터를 적용한 후에는 gpt-4 모델만 유지됩니다.

구성 목록은 다음과 같습니다:
```python
config_list = [
    {
        '모델': 'gpt-4',
        'api_key': '<귀하의 OpenAI API 키는 여기>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<귀하의 Azure OpenAI API 키는 여기>',
        'api_base': '<귀하의 Azure OpenAI API 베이스는 여기>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<귀하의 Azure OpenAI API 키는 여기>',
        'api_base': '<귀하의 Azure OpenAI API 베이스는 여기>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 '파일 업로드' 아이콘을 선택해 파일을 업로드할 수 있습니다.

원하는 경우 YAML 파일에서 불러오기 등 다른 방법으로 config_list의 값을 설정할 수도 있습니다.

## 에이전트 생성하기
이 단계에서는 먼저 티처블 에이전트의 메모리 저장소를 재설정하는 것으로 시작합니다. 이렇게 하면 디스크에 저장되어 있을 수 있는 이전 대화의 모든 메모리가 삭제됩니다.

In [2]:
from autogen.agentchat.contrib.teachable_agent import TeachableAgent
from autogen import UserProxyAgent

llm_config = {
    "request_timeout": 60,
    "config_list": config_list,
    "use_cache": True,  # Use False to explore LLM non-determinism.
}

teach_config={
    "verbosity": 0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    "reset_db": True,  # Set to True to start over with an empty database.
    "path_to_db_dir": "./tmp/notebook/teachable_agent_db",  # Path to the directory where the database will be stored.
    "recall_threshold": 1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
}

try:
    from termcolor import colored
except ImportError:
    def colored(x, *args, **kwargs):
        return x
    
autogen.ChatCompletion.start_logging()

teachable_agent = TeachableAgent(
    name="teachableagent",
    llm_config=llm_config,
    teach_config=teach_config)

user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=0,
)




CLEARING MEMORY


## 새로운 사실 학습하기
에이전트가 아직 알지 못하는 몇 가지 사실을 GPT-4의 학습 데이터보다 더 최근의 데이터로 학습시켜 보겠습니다.

In [3]:
text = "What is the Vicuna model?"
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachableagent):

What is the Vicuna model?

--------------------------------------------------------------------------------
teachableagent (to user):

I'm sorry, but I don't have information about the "Vicuna model" in my current database. Could you please provide some context, description, or details about this model so I can better assist you?

--------------------------------------------------------------------------------


In [4]:
text = "Vicuna is a 13B-parameter language model released by Meta."
user.initiate_chat(teachable_agent, message=text, clear_history=False)

user (to teachableagent):

Vicuna is a 13B-parameter language model released by Meta.

--------------------------------------------------------------------------------
teachableagent (to user):

Thank you for providing the information. The Vicuna model is a large-scale language model developed by Meta. It consists of 13 billion parameters, which is what makes it able to understand and generate human-like text across various topics. Parameters in a neural network are the weights and biases assigned to each connection between the neurons, and having more parameters enables the model to learn and capture more complex patterns and relationships in data. As an AI language model, Vicuna can be utilized in various language-related tasks, such as text completion, translation, summarization, and more.

--------------------------------------------------------------------------------


In [5]:
text = "What is the Orca model?"
user.initiate_chat(teachable_agent, message=text, clear_history=False)

user (to teachableagent):

What is the Orca model?

--------------------------------------------------------------------------------
teachableagent (to user):

The Orca model is a yet-to-be-released language model by Meta, referring to the Research Preview API release of Meta's advanced language model. While I don't have specific details about the Orca model at this time, it is expected that the model would have an even larger number of parameters to further enhance its capabilities compared to its predecessor, Vicuna.

As AI language models evolve, they generally become more efficient at understanding and generating human-like text. This makes them more effective in completing a wide range of tasks, including text completion, translation, summarization, and Q&A, as well as providing more accurate and coherent results in various applications.

--------------------------------------------------------------------------------


In [6]:
text = "Orca is a 13B-parameter language model released by Microsoft. It outperforms Vicuna on most tasks."
user.initiate_chat(teachable_agent, message=text, clear_history=False)

user (to teachableagent):

Orca is a 13B-parameter language model released by Microsoft. It outperforms Vicuna on most tasks.

--------------------------------------------------------------------------------
teachableagent (to user):

Thank you for the clarification. Orca is a 13-billion-parameter language model developed by Microsoft. It is designed to outperform Vicuna, the language model released by Meta. With a large number of parameters, Orca is capable of understanding and generating human-like text across diverse subjects, making it highly useful in various applications.

Orca's advanced capabilities allow it to excel in tasks such as text completion, translation, summarization, Q&A, and more. Its performance improvement over Vicuna highlights the rapid advancements in natural language processing (NLP) research and the development of increasingly capable AI language models.

--------------------------------------------------------------------------------


첫 번째 채팅은 여기서 끝내겠습니다. 각 채팅이 끝날 때마다 다음 함수를 호출하여 `TeachableAgent`가 사용자가 가르친 내용을 저장할 수 있도록 해야 합니다.

In [7]:
teachable_agent.learn_from_user_feedback()


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER


이제 이전 채팅의 기록을 지우고 새 채팅을 시작하겠습니다. 이 시점에서 일반적인 LLM 기반 어시스턴트는 마지막 채팅의 모든 내용을 잊어버립니다. 하지만 `TeachableAgent`는 필요에 따라 벡터 DB에서 기억을 불러올 수 있기 때문에 이전 대화에서 사용자가 가르친 사실을 기억하고 추론할 수 있습니다.

In [8]:
text = "How does the Vicuna model compare to the Orca model?"
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachableagent):

How does the Vicuna model compare to the Orca model?

--------------------------------------------------------------------------------
teachableagent (to user):

The Vicuna and Orca models are both 13B-parameter language models released by Meta and Microsoft, respectively. In terms of performance, Orca has been reported to outperform Vicuna on most tasks. However, without detailed information about specific tasks and benchmarks, it is difficult to provide a more comprehensive comparison. Generally speaking, both models are advanced language models that aim to provide high-quality natural language understanding and generation, but Orca appears to have an edge in terms of overall performance.

--------------------------------------------------------------------------------


## 사용자 기본 설정 학습하기
이제 에이전트에게 몇 가지 기본 설정을 가르쳐 봅시다. 팀원들이 읽을 수 있도록 새 논문의 짧은 요약을 자주 게시하는데, 학습 가능한 에이전트가 이 작업을 더 빠르게 처리하도록 돕고 싶다고 가정해 보겠습니다.

In [9]:
text = """Please summarize this abstract.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in many example applications, with domains ranging from mathematics, coding, question answering, operations research, online decision-making, entertainment, etc.
"""
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachableagent):

Please summarize this abstract.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in

하지만 이는 구조화되지 않았습니다. 따라서 상담원에게 우리의 선호도를 알려주자.

In [10]:
text = """Please summarize this abstract. 
When I'm summarizing an abstract, I try to make the summary contain just three short bullet points:  the title, the innovation, and the key empirical results.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in many example applications, with domains ranging from mathematics, coding, question answering, operations research, online decision-making, entertainment, etc.
"""
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachableagent):

Please summarize this abstract. 
When I'm summarizing an abstract, I try to make the summary contain just three short bullet points:  the title, the innovation, and the key empirical results.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a gene

훨씬 나아졌지만 교육 가능한 상담원이 나중에 다른 문서에서 이러한 기본 설정을 기억할 수 있을까요? 새 채팅을 시작하여 알아보세요!

In [11]:
teachable_agent.learn_from_user_feedback()

text = """Please summarize this abstract.

Sparks of Artificial General Intelligence: Early experiments with GPT-4
Sébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, Johannes Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Yin Tat Lee, Yuanzhi Li, Scott Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, Yi Zhang
Artificial intelligence (AI) researchers have been developing and refining large language models (LLMs) that exhibit remarkable capabilities across a variety of domains and tasks, challenging our understanding of learning and cognition. The latest model developed by OpenAI, GPT-4, was trained using an unprecedented scale of compute and data. In this paper, we report on our investigation of an early version of GPT-4, when it was still in active development by OpenAI. We contend that (this early version of) GPT-4 is part of a new cohort of LLMs (along with ChatGPT and Google's PaLM for example) that exhibit more general intelligence than previous AI models. We discuss the rising capabilities and implications of these models. We demonstrate that, beyond its mastery of language, GPT-4 can solve novel and difficult tasks that span mathematics, coding, vision, medicine, law, psychology and more, without needing any special prompting. Moreover, in all of these tasks, GPT-4's performance is strikingly close to human-level performance, and often vastly surpasses prior models such as ChatGPT. Given the breadth and depth of GPT-4's capabilities, we believe that it could reasonably be viewed as an early (yet still incomplete) version of an artificial general intelligence (AGI) system. In our exploration of GPT-4, we put special emphasis on discovering its limitations, and we discuss the challenges ahead for advancing towards deeper and more comprehensive versions of AGI, including the possible need for pursuing a new paradigm that moves beyond next-word prediction. We conclude with reflections on societal influences of the recent technological leap and future research directions."""
user.initiate_chat(teachable_agent, message=text, clear_history=True)


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER
user (to teachableagent):

Please summarize this abstract.

Sparks of Artificial General Intelligence: Early experiments with GPT-4
Sébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, Johannes Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Yin Tat Lee, Yuanzhi Li, Scott Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, Yi Zhang
Artificial intelligence (AI) researchers have been developing and refining large language models (LLMs) that exhibit remarkable capabilities across a variety of domains and tasks, challenging our understanding of learning and cognition. The latest model developed by OpenAI, GPT-4, was trained using an unprecedented scale of compute and data. In this paper, we report on our investigation of an early version of GPT-4, when it was still in active development by OpenAI. We contend that (this early version of) GPT-4 is part of a new cohort of LLMs (along with ChatGPT and Google's PaLM for example) that exhibit mor

## 새로운 스킬 학습하기
마지막으로, 어려운 유형의 작업을 수행하기 위한 새로운 스킬을 가르쳐서 티처블 에이전트의 역량을 확장해 봅시다.

Sparks of AGI](https://arxiv.org/abs/2303.12712) 논문에서는 다음과 같은 수학 문제에 대해 GPT-4를 평가했는데, 32%만 풀 수 있었습니다.

In [12]:
text = """Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(teachable_agent, message=text, clear_history=True)

user (to teachableagent):

Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
teachableagent (to user):

Step 1: Identify the current equation and the target value on the right-hand side.
Current equation: 9 * 4 + 6 * 6 = 72
Target value: 99

Step 2: Determine what numbers can be changed on the left-hand side.
Possible changes: 9, 4, 6 (first occurrence), and 6 (second occurrence)

Step 3: Evaluate whether each change in the numbers on the left-hand side can result in the target value.
9 -> If changed, it would create a larger product.
4 -> If changed, the product would increase or decrease by more than the amount needed to reach 99.
6 (first occurrence) -> If inc

이번에는 상담원에게 이러한 작업을 해결하기 위한 신뢰할 수 있는 전략을 알려주세요.

In [13]:
text = """Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".

Here's some advice that may help:
1. Let E denote the original number on the right.
2. Let F denote the final number on the right.
3. Calculate the difference between the two, G = F - E.
4. Examine the numbers on the left one by one until finding one that divides evenly into G, where negative integers are allowed.
5. Calculate J = G / H. This is the number of times that H divides into G.
6. Verify that J is an integer, and that H * J = G.
7. Find the number on the left which is multiplied by H, and call it K.
8. Change K to K + J.
9. Recompute the value on the left, and verify that it equals F.
Finally, write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(teachable_agent, message=text, clear_history=False)

user (to teachableagent):

Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".

Here's some advice that may help:
1. Let E denote the original number on the right.
2. Let F denote the final number on the right.
3. Calculate the difference between the two, G = F - E.
4. Examine the numbers on the left one by one until finding one that divides evenly into G, where negative integers are allowed.
5. Calculate J = G / H. This is the number of times that H divides into G.
6. Verify that J is an integer, and that H * J = G.
7. Find the number on the left which is multiplied by H, and call it K.
8. Change K to K + J.
9. Recompute the value on the left, and verify that it equals F.
Finally, write down your solution as: "The solution is: A * B + C * D".




이러한 조언이 주어지면 GPT-4는 95% 이상 이러한 문제를 해결할 수 있습니다. 하지만 학습 가능한 상담원이 전략을 기억하여 사용자가 반복해서 설명할 필요가 없도록 할 수 있을까요? 이전과 마찬가지로 새 채팅을 시작하여 알아봅시다.

In [14]:
teachable_agent.learn_from_user_feedback()

text = """Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(teachable_agent, message=text, clear_history=True)


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER
user (to teachableagent):

Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
teachableagent (to user):

Let's follow the advice to modify one integer on the left hand side of the equation to make the right hand side become 99:

1. Let E denote the original number on the right: E = 72.
2. Let F denote the final number on the right: F = 99.
3. Calculate the difference between the two, G = F - E: G = 99 - 72 = 27.
4. Examine the numbers on the left one by one to find one that divides evenly into G: 9, 4, 6. Since 9 divides 27 evenly, we will work with 9 (H = 9).
5. Calculate J = G / H: J = 27 / 9 = 3.
6. Verify that J i

최종 점검으로, 티처블 에이전트가 새로 학습한 스킬을 별도의 작업 인스턴스에서 테스트해 보겠습니다.

In [15]:
text = """Consider the identity:  
8 * 3 + 7 * 9 = 87
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 59?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(teachable_agent, message=text, clear_history=False)

user (to teachableagent):

Consider the identity:  
8 * 3 + 7 * 9 = 87
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 59?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
teachableagent (to user):

Let's follow the advice to modify one integer on the left hand side of the equation to make the right hand side become 59:

1. Let E denote the original number on the right: E = 87.
2. Let F denote the final number on the right: F = 59.
3. Calculate the difference between the two, G = F - E: G = 59 - 87 = -28.
4. Examine the numbers on the left one by one to find one that divides evenly into G: 8, 3, 7, 9. Since 7 divides -28 evenly, we will work with 7 (H = 7).
5. Calculate J = G / H: J = -28 / 7 = -4.
6. Verify that J is an integer, and that H * J = G: J is a